In [1]:
# Libraries Imports
import argparse
import json
from math import ceil
#import PIL
from PIL import Image
import torch
import numpy as np
from torchvision import datasets, transforms, models
from sklearn.metrics import confusion_matrix, classification_report
import torch.nn.functional as F
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Means and standard deviations for normalize images
mean = [0.485, 0.456, 0.406]
std_dev = [0.229, 0.224, 0.225]

# Sign Traffic Classes
class_names = ['give_way', 'no_entry', 'priority_road',  'stop']


In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    transf = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std_dev)
                                ])
   #img = Image.open(image)
    img_transf = transf(image)
    # TODO: Process a PIL image for use in a PyTorch model
    return img_transf


Getting Probability Prediction

In [3]:
def predict_proba(model, image_path):
  img = Image.open(image_path)
  img = process_image(img).unsqueeze(0)
  
  pred = model(img.to(device))
  pred = F.softmax(pred, dim=1)
  return pred.detach().cpu().numpy().flatten()

  
def show_prediction_confidence(prediction, class_names):
  pred_df = pd.DataFrame({
    'class_names': class_names,
    'values': prediction
  })
  sns.barplot(x='values', y='class_names', data=pred_df, orient='h')
  plt.xlim([0, 1]);
  

In [7]:
# Setting device available (gpu or cpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loading Model
model = torch.load('checkpoint.pth', map_location = device)
#model.eval()
#model 

RuntimeError: version_ <= kMaxSupportedFileFormatVersion INTERNAL ASSERT FAILED at /pytorch/caffe2/serialize/inline_container.cc:132, please report a bug to PyTorch. Attempted to read a PyTorch file with version 3, but the maximum supported version for reading is 2. Your PyTorch installation may be too old. (init at /pytorch/caffe2/serialize/inline_container.cc:132)
frame #0: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x33 (0x7f13cc4a9193 in /home/cristiansouza/anaconda3/lib/python3.7/site-packages/torch/lib/libc10.so)
frame #1: caffe2::serialize::PyTorchStreamReader::init() + 0x1f5b (0x7f13a6edcf8b in /home/cristiansouza/anaconda3/lib/python3.7/site-packages/torch/lib/libtorch.so)
frame #2: caffe2::serialize::PyTorchStreamReader::PyTorchStreamReader(std::string const&) + 0x64 (0x7f13a6ede1a4 in /home/cristiansouza/anaconda3/lib/python3.7/site-packages/torch/lib/libtorch.so)
frame #3: <unknown function> + 0x68ea06 (0x7f13b43a1a06 in /home/cristiansouza/anaconda3/lib/p

In [6]:
??torch.load

+ Read your test dataset, 
+ get the prediction of your test dataset images 
+ based on real and predicted values, generate the metrics that you consider useful and to evaluate the model, explaining why you considered that.
+ Explain the results of the test.  
